In [1]:
## First load the necessary modules. 

import skimage
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [ ]:
##Load the data and prep it. 

Cells=np.load("/content/drive/My Drive/Cells.npy")
labels=np.load("/content/drive/My Drive/labels.npy")

In [ ]:
from sklearn.model_selection import train_test_split  

#Stratification used for labels and split is 80/20. 
X_train, X_test, y_train, y_test = train_test_split(Cells, labels, test_size = 0.2)

In [ ]:
# Conversion from utf-8
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32') 

X_train = X_train/255.0
X_test = X_test/255.0

y_train=keras.utils.to_categorical(y_train, 2)
y_test=keras.utils.to_categorical(y_test, 2)

In [ ]:
## Load the imports. 

import keras
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras import models
from tensorflow.keras import layers
from keras.models import load_model

In [ ]:
#Load the pretrained student and teacher model. 

In [ ]:
student = load_model('/content/drive/MyDrive/student.h5')

In [ ]:
teacher = load_model('/content/drive/MyDrive/teacher.h5')

In [ ]:
## To check if the teacher and student have the last softmax layer activated. 
teacher.summary()

In [ ]:
## If not then add softmax layer and compile it. 

teacher.add(Activation('softmax', name='Softmax')) 

teacher.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[keras.metrics.CategoricalAccuracy()],
)

In [ ]:
## Same for student. 

student.add(Activation('softmax')) 
student.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[keras.metrics.CategoricalAccuracy()],
)

In [ ]:
IMGS = 64


In [ ]:
## Setting the layers. 
lay = []
for layer in teacher.layers:
    lay.append(layer.name)
    print(layer.name)

In [ ]:
lay_s = []
for layer in student.layers:
    lay_s.append(layer.name)
    print(layer.name)

In [ ]:
t_last_conv_layer_name = lay[4]

t_classifier_layer_names = lay[5:]

s_last_conv_layer_name = lay_s[2]

s_classifier_layer_names = lay_s[3:]

In [ ]:
## Defining the functions. 

def get_img_array(img_path, size):
    np.seterr(divide='ignore')
    img_arr = cv2.imread(img_path)
    #img_arr = crop_brain_contour(img_arr, False)
    img_arr = cv2.resize(img_arr, (IMGS, IMGS))
    img_arr = img_arr.reshape(1, IMGS, IMGS, 3)
    return img_arr

In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, classifier_layer_names):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer
    last_conv_layer = model.get_layer(last_conv_layer_name)
    last_conv_layer_model = tf.keras.Model(model.inputs, last_conv_layer.output)

    # Second, we create a model that maps the activations of the last conv
    # layer to the final class predictions
    classifier_input = tf.keras.Input(shape = last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)

    classifier_model = keras.engine.Model(classifier_input, x)

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it

        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)

        prop_grad = last_conv_layer_output # propagate all layers inside the tape block
        for layer in classifier_layer_names[:-2]: # all massive without last element
            prop_grad = model.get_layer(layer)(prop_grad)

        # Compute class predictions
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    # This is the gradient of the top predicted class with regard to
    # the output feature map of the last conv layer

    grads = tape.gradient(prop_grad, last_conv_layer_output) 
    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]
    last_conv_layer_output = last_conv_layer_output*pooled_grads

    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    heatmap = np.mean(last_conv_layer_output, axis=-1)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    # heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap

In [ ]:
## Defining the images. 

img_path1 = '/content/drive/MyDrive/Normal_cell2.png'

img_path2 = '/content/drive/MyDrive/cell_181.png'

In [ ]:
model_builder = teacher
img_size = (IMGS, IMGS)
classes = ['Infected','NORMAL']

In [ ]:
img_array1 = get_img_array(img_path1, size = img_size)
img_array2 = get_img_array(img_path2, size = img_size)

tpreds1 = teacher.predict(img_array1)
tpreds2 = teacher.predict(img_array2)

spreds1 = student.predict(img_array1)
spreds2 = student.predict(img_array2)

print(tpreds1)
print("Predicted:", classes[int(np.round(teacher.predict(img_array1)[0][0]))])

# Generate class activation heatmap for healthy cell
heatmap1 = make_gradcam_heatmap(
    img_array1, teacher, t_last_conv_layer_name, t_classifier_layer_names
)
# Display heatmap
plt.matshow(heatmap1)
plt.savefig('heatmap_teacher_healthy.png')
plt.show()

In [ ]:
#Generate heatmap for healthy cell
heatmap2 = make_gradcam_heatmap(img_array1,
      student, s_last_conv_layer_name, s_classifier_layer_names)

# Display heatmap
plt.matshow(heatmap2)
plt.savefig('heatmap_student_healthy.png')
plt.show()

In [ ]:
## Carry out SSIM, here it is done only for two images. 

In [ ]:
teacher_img = cv2.imread("/content/heatmap_teacher_healthy_8.png")
stu_img = cv2.imread("/content/heatmap_student_healthy_8.png")

In [ ]:
teacher_img = cv2.cvtColor(teacher_img, cv2.COLOR_BGR2GRAY)
stu_img = cv2.cvtColor(stu_img, cv2.COLOR_BGR2GRAY)

In [ ]:
s = ssim(teacher_img, stu_img)
print(s)